In [ ]:
# Core PyTorch libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Computer vision utilities
import torchvision
import torchvision.transforms as transforms
from torchvision import models

# Data handling and visualization
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import copy

# Machine learning utilities
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# Configure matplotlib for high-quality plots
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10
plt.style.use('default')

print("✅ Libraries imported successfully!")
print(f"📦 PyTorch version: {torch.__version__}")
print(f"🖼️ Torchvision version: {torchvision.__version__}")
print(f"🔥 CUDA available: {torch.cuda.is_available()}")
print(f"🍎 MPS available: {torch.backends.mps.is_available()}")

# Check MobileNet-V2 availability
try:
    test_model = models.mobilenet_v2(pretrained=False)
    print("✅ MobileNet-V2 available!")
    print(f"🔢 Model parameters: {sum(p.numel() for p in test_model.parameters()):,}")
    del test_model  # Clean up
except Exception as e:
    print(f"❌ MobileNet-V2 not available: {e}")
    print("💡 Please update PyTorch/torchvision to latest version")

# Set device (MobileNet-V2 works well on CPU too!)
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"🚀 Using GPU: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("🚀 Using Apple MPS")
else:
    print("💻 Using CPU (perfectly fine for MobileNet-V2!)")

print(f"🎯 Device selected: {device}")

# Memory optimization (less critical for MobileNet-V2)
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"🧹 GPU memory cleared")
    print(f"💾 GPU memory available: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")


In [ ]:
# Define data transformations for MobileNet-V2 (224×224 input)
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),  # Slightly larger for crop
    transforms.RandomCrop(224),     # Standard resolution
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Direct resize for validation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets
print("📁 Loading Flowers102 dataset...")
try:
    train_dataset = torchvision.datasets.Flowers102(
        root='./data', 
        split='train',
        transform=train_transforms,
        download=True
    )
    
    val_dataset = torchvision.datasets.Flowers102(
        root='./data', 
        split='val',
        transform=val_transforms,
        download=False
    )
    
    test_dataset = torchvision.datasets.Flowers102(
        root='./data', 
        split='test',
        transform=val_transforms,
        download=False
    )
    
    print(f"✅ Dataset loaded successfully!")
    print(f"📊 Training images: {len(train_dataset)}")
    print(f"📊 Validation images: {len(val_dataset)}")
    print(f"📊 Test images: {len(test_dataset)}")
    
except Exception as e:
    print(f"❌ Error loading dataset: {e}")
    print("💡 Make sure you have internet connection for first download")

# Create data loaders - MobileNet-V2 can handle larger batch sizes
BATCH_SIZE = 32  # Efficient batch size for MobileNet-V2

train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    num_workers=0,
    pin_memory=torch.cuda.is_available()
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False,
    num_workers=0,
    pin_memory=torch.cuda.is_available()
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False,
    num_workers=0,
    pin_memory=torch.cuda.is_available()
)

print(f"🔄 Data loaders created with batch size: {BATCH_SIZE}")
print(f"📦 Training batches: {len(train_loader)}")
print(f"📦 Validation batches: {len(val_loader)}")
print(f"📦 Test batches: {len(test_loader)}")

# Memory usage comparison
print(f"\n💾 Memory usage comparison:")
print(f"   EfficientNet-B3 (300×300): {24 * 3 * 300 * 300 / 1024**2:.1f}MB per batch")
print(f"   MobileNet-V2 (224×224): {32 * 3 * 224 * 224 / 1024**2:.1f}MB per batch")

# Visualize sample images
def visualize_batch(data_loader, title):
    """Visualize a batch of images"""
    batch_images, batch_labels = next(iter(data_loader))
    
    # Denormalize for visualization
    mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)
    batch_images = batch_images * std + mean
    
    fig, axes = plt.subplots(2, 4, figsize=(12, 6))
    axes = axes.flatten()
    
    for i in range(8):
        img = batch_images[i].permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[i].imshow(img)
        axes[i].set_title(f'Class: {batch_labels[i].item()}')
        axes[i].axis('off')
    
    plt.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Visualize training samples
print("\n🖼️ Sample training images:")
visualize_batch(train_loader, "Training Images (224×224, Mobile-Optimized)")

# Dataset statistics
print(f"\n📊 Dataset Statistics:")
print(f"   Input Resolution: 224×224 (efficient for mobile)")
print(f"   Total Classes: 102")
print(f"   Training Images: {len(train_dataset)}")
print(f"   Validation Images: {len(val_dataset)}")
print(f"   Test Images: {len(test_dataset)}")
print(f"   Images per Class: ~{len(train_dataset) / 102:.1f} (train), ~{len(val_dataset) / 102:.1f} (val)")

print(f"\n🚀 Mobile Advantages:")
print(f"   ✅ Standard resolution (224×224) - mobile-friendly")
print(f"   ✅ Efficient batch size (32) - better gradient estimates")
print(f"   ✅ Lower memory usage - works on any device")
print(f"   ✅ Fast data loading - optimized pipeline")


In [ ]:
# Load pre-trained MobileNet-V2 model
print("🏗️ Loading MobileNet-V2 model...")
model = models.mobilenet_v2(pretrained=True)

# Modify the classifier for 102 flower classes
num_classes = 102
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# Move model to device
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✅ MobileNet-V2 model loaded successfully!")
print(f"📊 Total parameters: {total_params:,}")
print(f"🎯 Trainable parameters: {trainable_params:,}")
print(f"📱 Model size: ~{total_params * 4 / 1024**2:.1f}MB")

# Analyze model architecture
print(f"\n🔍 MobileNet-V2 Architecture Analysis:")
print(f"   Input Resolution: 224×224")
print(f"   Feature Extractor: Inverted Residual Blocks")
print(f"   Classifier: Adaptive Avg Pool + Linear")
print(f"   Output Classes: {num_classes}")

# Compare with other models
print(f"\n📊 Model Comparison:")
print(f"   ResNet18: 11.7M parameters")
print(f"   ResNet50: 25.6M parameters")
print(f"   EfficientNet-B0: 5.3M parameters")
print(f"   EfficientNet-B3: 12.2M parameters")
print(f"   MobileNet-V2: {total_params/1000000:.1f}M parameters ⭐")

# Visualize model architecture
print(f"\n🏗️ Model Architecture:")
print(model)

# Analyze feature extractor layers
print(f"\n🔬 Feature Extractor Analysis:")
feature_layers = 0
for name, module in model.features.named_modules():
    if isinstance(module, nn.Conv2d):
        feature_layers += 1
        
print(f"   Total Conv2d layers: {feature_layers}")
print(f"   Inverted Residual blocks: 17")
print(f"   Depthwise separable convs: Most operations")
print(f"   Skip connections: When stride=1")

# Test forward pass
print(f"\n🧪 Testing forward pass...")
with torch.no_grad():
    sample_input = torch.randn(1, 3, 224, 224).to(device)
    sample_output = model(sample_input)
    print(f"✅ Forward pass successful!")
    print(f"   Input shape: {sample_input.shape}")
    print(f"   Output shape: {sample_output.shape}")
    print(f"   Output classes: {sample_output.shape[1]}")

# Memory usage analysis
if device.type == 'cuda':
    print(f"\n💾 GPU Memory Usage:")
    print(f"   Allocated: {torch.cuda.memory_allocated(0)/1024**2:.1f}MB")
    print(f"   Cached: {torch.cuda.memory_reserved(0)/1024**2:.1f}MB")
    
print(f"\n🚀 MobileNet-V2 Advantages:")
print(f"   ✅ Smallest model in our comparison")
print(f"   ✅ Efficient depthwise separable convolutions")
print(f"   ✅ Inverted residual blocks for better gradient flow")
print(f"   ✅ Optimized for mobile deployment")
print(f"   ✅ Fast training and inference")
print(f"   ✅ Low memory footprint")

# Efficiency metrics
print(f"\n📈 Efficiency Metrics:")
print(f"   Parameters: {total_params:,}")
print(f"   Model Size: ~{total_params * 4 / 1024**2:.1f}MB")
print(f"   Expected Accuracy: ~87%")
print(f"   Efficiency: {87/(total_params/1000000):.1f}% per M params")
print(f"   Training Time: ~8-12 minutes")
print(f"   Mobile Score: ⭐⭐⭐⭐⭐")


In [ ]:
# Training configuration
NUM_EPOCHS = 50
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.01

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer (AdamW works well for mobile architectures)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

print("⚙️ Training Configuration:")
print(f"   Total Epochs: {NUM_EPOCHS}")
print(f"   Initial Learning Rate: {LEARNING_RATE}")
print(f"   Weight Decay: {WEIGHT_DECAY}")
print(f"   Batch Size: {BATCH_SIZE}")
print(f"   Optimizer: AdamW")
print(f"   Scheduler: StepLR (step_size=20, gamma=0.1)")
print(f"   Loss Function: CrossEntropyLoss")

# Training phase configuration
print(f"\n📚 Training Phases:")
print(f"   Phase 1 (Epochs 1-20): Feature extraction (frozen backbone)")
print(f"   Phase 2 (Epochs 21-50): Fine-tuning (unfrozen backbone)")

# Mobile-specific optimizations
print(f"\n📱 Mobile Optimizations:")
print(f"   ✅ Efficient batch size (32) for faster training")
print(f"   ✅ Lower learning rates to preserve mobile optimizations")
print(f"   ✅ AdamW optimizer for stable convergence")
print(f"   ✅ Progressive training strategy")

# Memory efficiency analysis
print(f"\n💾 Memory Efficiency:")
print(f"   Model Size: ~{total_params * 4 / 1024**2:.1f}MB")
print(f"   Batch Memory: ~{BATCH_SIZE * 3 * 224 * 224 * 4 / 1024**2:.1f}MB")
print(f"   Total GPU Memory: ~{(total_params * 4 + BATCH_SIZE * 3 * 224 * 224 * 4) / 1024**2:.1f}MB")

# Helper functions for training
def freeze_backbone(model):
    """Freeze feature extractor for phase 1"""
    for param in model.features.parameters():
        param.requires_grad = False
    # Only classifier parameters are trainable
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"🔒 Backbone frozen. Trainable parameters: {trainable_params:,}")

def unfreeze_backbone(model):
    """Unfreeze all layers for phase 2"""
    for param in model.parameters():
        param.requires_grad = True
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"🔓 Backbone unfrozen. Trainable parameters: {trainable_params:,}")

def train_epoch(model, train_loader, criterion, optimizer, device, epoch):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch}')
    
    for batch_idx, (inputs, targets) in enumerate(progress_bar):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        # Update progress bar
        progress_bar.set_postfix({
            'Loss': f'{running_loss/(batch_idx+1):.3f}',
            'Acc': f'{100.*correct/total:.2f}%'
        })
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc

def validate_epoch(model, val_loader, criterion, device):
    """Validate for one epoch"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc='Validation')
        
        for batch_idx, (inputs, targets) in enumerate(progress_bar):
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            progress_bar.set_postfix({
                'Loss': f'{running_loss/(batch_idx+1):.3f}',
                'Acc': f'{100.*correct/total:.2f}%'
            })
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc

print(f"\n🚀 Ready to start training!")
print(f"   Expected training time: ~8-12 minutes")
print(f"   Expected Phase 1 accuracy: ~80%")
print(f"   Expected Phase 2 accuracy: ~87%")
print(f"   Mobile efficiency: 24.9% per M params")


In [ ]:
# Phase 1: Feature Extraction Training (Epochs 1-20)
print("🚀 Starting Phase 1: Feature Extraction Training")
print("="*60)

# Freeze backbone for feature extraction
freeze_backbone(model)

# Initialize tracking variables
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
epoch_times = []

# Best model tracking
best_val_acc = 0.0
best_model_state = None

# Phase 1 training loop
phase1_start_time = time.time()

for epoch in range(1, 21):  # Epochs 1-20
    epoch_start_time = time.time()
    
    # Training
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, epoch)
    
    # Validation
    val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
    
    # Update learning rate
    scheduler.step()
    
    # Track metrics
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    
    epoch_time = time.time() - epoch_start_time
    epoch_times.append(epoch_time)
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = copy.deepcopy(model.state_dict())
    
    # Print epoch summary
    print(f"Epoch {epoch:2d}/20 | "
          f"Train Loss: {train_loss:.3f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.3f} | Val Acc: {val_acc:.2f}% | "
          f"Time: {epoch_time:.1f}s")
    
    # Early stopping check (optional)
    if epoch > 5 and val_acc < max(val_accuracies[:-1]) - 5:
        print(f"⚠️  Early stopping triggered at epoch {epoch}")
        break

phase1_end_time = time.time()
phase1_duration = phase1_end_time - phase1_start_time

print("\n" + "="*60)
print("📊 Phase 1 Training Complete!")
print("="*60)
print(f"🎯 Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"⏱️  Total Training Time: {phase1_duration:.1f} seconds ({phase1_duration/60:.1f} minutes)")
print(f"⚡ Average Time per Epoch: {phase1_duration/20:.1f} seconds")
print(f"🏆 Expected vs Actual: ~80% vs {best_val_acc:.2f}%")

# Performance analysis
phase1_final_train_acc = train_accuracies[-1]
phase1_final_val_acc = val_accuracies[-1]

print(f"\n📈 Phase 1 Performance Analysis:")
print(f"   Final Training Accuracy: {phase1_final_train_acc:.2f}%")
print(f"   Final Validation Accuracy: {phase1_final_val_acc:.2f}%")
print(f"   Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"   Overfitting Gap: {phase1_final_train_acc - phase1_final_val_acc:.2f}%")

# Mobile efficiency metrics
print(f"\n📱 Mobile Efficiency Metrics:")
print(f"   Training Speed: {len(train_dataset) / phase1_duration:.1f} images/second")
print(f"   Parameter Efficiency: {best_val_acc / (total_params/1000000):.1f}% per M params")
print(f"   Memory Efficiency: Low memory usage due to frozen backbone")

# Prepare for Phase 2
print(f"\n🔄 Preparing for Phase 2...")
print(f"   Phase 1 baseline: {best_val_acc:.2f}%")
print(f"   Phase 2 target: ~87% (+{87-best_val_acc:.1f}% improvement)")
print(f"   Strategy: Unfreeze backbone, lower learning rate")

# Load best model from Phase 1
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"✅ Loaded best model from Phase 1 (Val Acc: {best_val_acc:.2f}%)")

print(f"\n🚀 Ready for Phase 2: Fine-tuning!")
print(f"   Expected improvement: ~{87-best_val_acc:.1f}%")
print(f"   Expected final accuracy: ~87%")
print(f"   Mobile advantages: Efficient training, fast convergence")


In [ ]:
# Phase 2: Fine-tuning Training (Epochs 21-50)
print("\n🔄 Starting Phase 2: Fine-tuning Training")
print("="*60)

# Unfreeze backbone for fine-tuning
unfreeze_backbone(model)

# Reset optimizer with lower learning rate for fine-tuning
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

print(f"📉 Learning rate reduced to 0.0001 for fine-tuning")

# Phase 2 training loop
phase2_start_time = time.time()

for epoch in range(21, 51):  # Epochs 21-50
    epoch_start_time = time.time()
    
    # Training
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, epoch)
    
    # Validation
    val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
    
    # Update learning rate
    scheduler.step()
    
    # Track metrics
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    
    epoch_time = time.time() - epoch_start_time
    epoch_times.append(epoch_time)
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = copy.deepcopy(model.state_dict())
        print(f"🌟 New best model! Validation accuracy: {val_acc:.2f}%")
    
    # Print epoch summary
    print(f"Epoch {epoch:2d}/50 | "
          f"Train Loss: {train_loss:.3f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.3f} | Val Acc: {val_acc:.2f}% | "
          f"Time: {epoch_time:.1f}s")
    
    # Early stopping check (optional)
    if epoch > 30 and val_acc < max(val_accuracies[-5:]) - 2:
        print(f"⚠️ Early stopping triggered at epoch {epoch}")
        break

phase2_end_time = time.time()
phase2_duration = phase2_end_time - phase2_start_time
total_training_time = phase1_duration + phase2_duration

print("\n" + "="*60)
print("🎉 Phase 2 Training Complete!")
print("="*60)
print(f"🎯 Final Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"⏱️ Phase 2 Training Time: {phase2_duration:.1f} seconds ({phase2_duration/60:.1f} minutes)")
print(f"⏱️ Total Training Time: {total_training_time:.1f} seconds ({total_training_time/60:.1f} minutes)")
print(f"🏆 Expected vs Actual: ~87% vs {best_val_acc:.2f}%")

# Performance improvement analysis
phase1_best = max(val_accuracies[:20]) if len(val_accuracies) >= 20 else best_val_acc
phase2_improvement = best_val_acc - phase1_best

print(f"\n📈 Training Progress Analysis:")
print(f"   Phase 1 Best: {phase1_best:.2f}%")
print(f"   Phase 2 Best: {best_val_acc:.2f}%")
print(f"   Improvement: +{phase2_improvement:.2f}%")
print(f"   Target Achievement: {best_val_acc/87*100:.1f}% of 87% target")

# Final performance metrics
final_train_acc = train_accuracies[-1]
final_val_acc = val_accuracies[-1]

print(f"\n📊 Final Performance Metrics:")
print(f"   Final Training Accuracy: {final_train_acc:.2f}%")
print(f"   Final Validation Accuracy: {final_val_acc:.2f}%")
print(f"   Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"   Overfitting Gap: {final_train_acc - final_val_acc:.2f}%")

# Mobile efficiency analysis
print(f"\n📱 Mobile Efficiency Analysis:")
print(f"   Total Parameters: {total_params:,}")
print(f"   Model Size: ~{total_params * 4 / 1024**2:.1f}MB")
print(f"   Accuracy per M Parameters: {best_val_acc/(total_params/1000000):.1f}%/M")
print(f"   Training Speed: {len(train_dataset) * NUM_EPOCHS / total_training_time:.1f} images/second")
print(f"   Inference Speed: Expected ~2-3ms per image")

# Load best model for final evaluation
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"\n✅ Loaded best model for final evaluation")
    print(f"   Best Validation Accuracy: {best_val_acc:.2f}%")

# Model comparison summary
print(f"\n🏆 MobileNet-V2 vs Other Models:")
print(f"   ResNet18: ~85% accuracy, 11.7M params")
print(f"   ResNet50: ~88% accuracy, 25.6M params") 
print(f"   EfficientNet-B0: ~90% accuracy, 5.3M params")
print(f"   EfficientNet-B3: ~92% accuracy, 12.2M params")
print(f"   MobileNet-V2: {best_val_acc:.1f}% accuracy, {total_params/1000000:.1f}M params ⭐")

print(f"\n🚀 Mobile Deployment Advantages:")
print(f"   ✅ Smallest model ({total_params/1000000:.1f}M parameters)")
print(f"   ✅ Fastest training ({total_training_time/60:.1f} minutes)")
print(f"   ✅ Highest efficiency ({best_val_acc/(total_params/1000000):.1f}% per M params)")
print(f"   ✅ Mobile-optimized architecture")
print(f"   ✅ Real-time inference capability")
print(f"   ✅ Low memory footprint")

print(f"\n🎯 Training Complete! Ready for deployment analysis...")


In [ ]:
# Plot comprehensive training results for MobileNet-V2
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
epochs = range(1, len(train_losses) + 1)

# Loss curves
axes[0, 0].plot(epochs, train_losses, 'b-', label='Training Loss', linewidth=2)
axes[0, 0].plot(epochs, val_losses, 'r-', label='Validation Loss', linewidth=2)
axes[0, 0].axvline(x=20, color='gray', linestyle='--', alpha=0.7, label='Phase Transition')
axes[0, 0].set_title('Training and Validation Loss (MobileNet-V2)', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Accuracy curves
axes[0, 1].plot(epochs, train_accuracies, 'b-', label='Training Accuracy', linewidth=2)
axes[0, 1].plot(epochs, val_accuracies, 'r-', label='Validation Accuracy', linewidth=2)
axes[0, 1].axvline(x=20, color='gray', linestyle='--', alpha=0.7, label='Phase Transition')
axes[0, 1].set_title('Training and Validation Accuracy (MobileNet-V2)', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy (%)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Training time per epoch
axes[1, 0].plot(epochs, epoch_times, 'g-', linewidth=2)
axes[1, 0].axvline(x=20, color='gray', linestyle='--', alpha=0.7, label='Phase Transition')
axes[1, 0].set_title('Training Time per Epoch (MobileNet-V2)', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Time (seconds)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Complete model comparison - efficiency vs accuracy
models = ['ResNet18', 'ResNet50', 'EfficientNet-B0', 'EfficientNet-B3', 'MobileNet-V2']
params = [11.7, 25.6, 5.3, 12.2, 3.5]  # Million parameters
accuracies = [85, 88, 90, 92, best_val_acc]  # Best validation accuracies
colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold', 'red']
sizes = [100, 150, 120, 140, 200]  # Different sizes for emphasis

# Create scatter plot: parameters vs accuracy
scatter = axes[1, 1].scatter(params, accuracies, c=colors, s=sizes, alpha=0.7, edgecolors='black')
axes[1, 1].set_title('Final Model Comparison: Parameters vs Accuracy', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Parameters (Millions)')
axes[1, 1].set_ylabel('Accuracy (%)')
axes[1, 1].set_xlim(0, 30)
axes[1, 1].set_ylim(80, 95)

# Add model labels with efficiency
for i, model in enumerate(models):
    efficiency = accuracies[i] / params[i]
    axes[1, 1].annotate(f'{model}\n({efficiency:.1f}%/M)', 
                       (params[i], accuracies[i]), 
                       xytext=(5, 5), textcoords='offset points', 
                       fontsize=8, ha='left')

axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print comprehensive final results
print("\n" + "="*80)
print("MOBILENET-V2 TRANSFER LEARNING FINAL RESULTS")
print("="*80)
print(f"📊 Final Validation Accuracy: {best_val_acc:.2f}%")
print(f"📈 Expected: ~87%, Achieved: {best_val_acc:.2f}%")
print(f"⏱️ Total Training Time: {total_training_time:.1f} seconds ({total_training_time/60:.1f} minutes)")
print(f"🔧 Model Parameters: {total_params:,}")
print(f"📱 Model Size: ~{total_params * 4 / 1024**2:.1f}MB")

# Phase analysis
phase1_acc = val_accuracies[19] if len(val_accuracies) >= 20 else val_accuracies[-1]
phase2_acc = best_val_acc
print(f"\n📊 Phase Analysis:")
print(f"   Phase 1 (Feature Extraction): {phase1_acc:.2f}%")
print(f"   Phase 2 (Fine-tuning): {phase2_acc:.2f}%")
print(f"   Improvement: +{phase2_acc - phase1_acc:.2f}%")

# Complete model ranking by different metrics
print(f"\n🏆 FINAL MODEL RANKING ACROSS ALL METRICS:")

print(f"\n📊 By Accuracy (High to Low):")
model_accuracies = [
    ("EfficientNet-B3", 92, 12.2),
    ("EfficientNet-B0", 90, 5.3),
    ("ResNet50", 88, 25.6),
    ("MobileNet-V2", best_val_acc, 3.5),
    ("ResNet18", 85, 11.7)
]
for i, (model, acc, params) in enumerate(sorted(model_accuracies, key=lambda x: x[1], reverse=True), 1):
    print(f"   {i}. {model}: {acc:.1f}% ({params:.1f}M params)")

print(f"\n⚡ By Efficiency (% per M params, High to Low):")
model_efficiency = [(name, acc/params) for name, acc, params in model_accuracies]
for i, (model, eff) in enumerate(sorted(model_efficiency, key=lambda x: x[1], reverse=True), 1):
    print(f"   {i}. {model}: {eff:.1f}% per M params")

print(f"\n📱 By Mobile Deployment Score:")
print(f"   🥇 MobileNet-V2: {best_val_acc:.1f}% accuracy, {3.5:.1f}M params, {total_training_time/60:.1f}min")
print(f"   🥈 EfficientNet-B0: 90% accuracy, 5.3M params, ~12min")
print(f"   🥉 EfficientNet-B3: 92% accuracy, 12.2M params, ~22min")
print(f"   4th ResNet18: 85% accuracy, 11.7M params, ~18min")
print(f"   5th ResNet50: 88% accuracy, 25.6M params, ~28min")

# Mobile deployment analysis
print(f"\n📱 MOBILE DEPLOYMENT ANALYSIS:")
print(f"   🏆 Winner: MobileNet-V2")
print(f"   • Smallest model: {total_params/1000000:.1f}M parameters")
print(f"   • Fastest training: {total_training_time/60:.1f} minutes")
print(f"   • Best efficiency: {best_val_acc/(total_params/1000000):.1f}% per M params")
print(f"   • Real-time inference: ~2-3ms per image")
print(f"   • Memory footprint: ~{total_params * 4 / 1024**2:.1f}MB")
print(f"   • Battery friendly: Optimized for mobile CPUs")

# Architecture innovation impact
print(f"\n🔬 ARCHITECTURE INNOVATION IMPACT:")
print(f"   💡 Depthwise Separable Convolutions:")
print(f"     • Parameter reduction: 88% fewer than standard convolutions")
print(f"     • Computational efficiency: 8-9× fewer multiply-adds")
print(f"     • Memory efficiency: Lower intermediate activations")
print(f"   💡 Inverted Residual Blocks:")
print(f"     • Efficient feature reuse with expand-depthwise-project")
print(f"     • Stable gradient flow for mobile architectures")
print(f"     • Linear bottlenecks prevent information loss")

# Practical deployment recommendations
print(f"\n🚀 PRACTICAL DEPLOYMENT RECOMMENDATIONS:")
print(f"   📱 Use Cases:")
print(f"     • Mobile flower identification apps")
print(f"     • Edge device botanical monitoring")
print(f"     • IoT garden management systems")
print(f"     • Real-time plant classification")
print(f"   ⚙️ Optimization Strategies:")
print(f"     • Quantization: INT8 for 4× inference speedup")
print(f"     • Pruning: Remove redundant connections")
print(f"     • Knowledge distillation: Train even smaller models")
print(f"     • Hardware acceleration: Utilize mobile NPUs")

# Course completion summary
print(f"\n🎓 TRANSFER LEARNING COURSE COMPLETION SUMMARY:")
print(f"   ✅ 5 Model Architectures Mastered:")
print(f"     1. ResNet18: Residual learning foundations")
print(f"     2. ResNet50: Deep network scaling")
print(f"     3. EfficientNet-B0: Compound scaling introduction")
print(f"     4. EfficientNet-B3: Systematic scaling mastery")
print(f"     5. MobileNet-V2: Mobile optimization excellence")
print(f"   ✅ Key Concepts Learned:")
print(f"     • Transfer learning strategies")
print(f"     • Progressive training techniques")
print(f"     • Architecture trade-offs")
print(f"     • Mobile deployment considerations")
print(f"     • Efficiency optimization methods")

print("="*80)
print("🎉 CONGRATULATIONS! Transfer Learning Course Complete! 🎉")
print("="*80)
